In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import stpsf

# from scipy.optimize import curve_fit
from astropy.io import fits
from astropy.convolution import convolve_fft
from joblib import Parallel, delayed
from astropy.stats import sigma_clipped_stats
from photutils.detection import StarFinder
# from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, aperture_photometry

%matplotlib widget

import warnings
warnings.filterwarnings("ignore")

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-068 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-071
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


In [2]:
def linear_fitting(coords,cube,n_int,n_group):
    """
    fits ramps piecewise with 2 straight lines if a jump is detected,
    otherwise fits a single line. If a piece is too short, fits only the longer one.
    """
    row, col = coords

    grads = [] # straight line gradient based on first data points
    intercepts = [] # straight line intercept based on first data points
    resids = [] # residuals from curve_fit of power law

    for int_num in range(n_int):
        x_dat = [i + (int_num)*n_group for i in range(n_group)]
        y_dat = [cube[x][row][col] for x in x_dat]
        x = np.asarray(x_dat, dtype=float) 
        y = np.asarray(y_dat, dtype=float)

        # removing first and last frames
        y[0] = np.nan
        y[-1] = np.nan

        mask = ~np.isnan(y)
        x = x[mask]
        y = y[mask]
        

        p,r,*_ = np.polyfit(x,y,1,full=True)
        m = p[0]
        c = p[1]
        grads.append(m)
        intercepts.append(c)
        resids.append(r[0] if len(r) > 0 else np.nan)
        
    return [row, col, grads, intercepts, resids]

In [ ]:
class Jurassic():
    """
        Class for searching the ramps of full array MIRI images for fast transients

        JURASSIC: JWST Up the Ramp Analysis Searching the Sky for Infrared Transients
    """

    def __init__(self,file=None,ramps=True,images=True,run=True):
        """
        Initialise or whatevs

        Parameters
        ----------
        file : str
                File name of the observation
        

        other stuff I guess
        """

        self.file = file
        
        if run:
            self._assign_data()
            self._make_cubes()
            self._mask_pixels()

            if ramps: # search on ramp level
                self.parallel_fit_df(self.rampy_cube) # only fitting rampy_cube not mega
                self.worst_fits(self.rampy_cube) # atm has 25 worst fitting pixels
        
            if images: # search on image level
                self.mega_inator(self.rampy_cube)
                self._cube_gradient(self.mega_cube_masked)
                self._reference_frame()
                self._cube_differenced()
                self._psf_kernel()
                self.source_finding()
                self._counts_to_df()

                self._cube_significance()
                self._cube_threshold() 
                self._cube_rolling_sum()



    def _assign_data(self):
        """
        Opens the fits file and assigns the data to the class
        """
        with fits.open(self.file) as hdul:
            self.data = hdul[1].data # science data
            self.dq_2d_arr = hdul[2].data # data quality flag array for whole cube
            self.dq_3d_arr = hdul[3].data # data quality flag array for each group
            self.filter = hdul[0].header['FILTER']

        self.n_int = len(self.data)
        self.n_group = len(self.data[0])
        self.n_frame = self.n_int * self.n_group


    def _make_cubes(self):
        """
        makes cube from 4d uncal file, also empty quality cube
        """
        # make the rampy science data cube
        ramps = np.array_split(self.data,self.n_int,axis=0)
        rampy_cube = np.concatenate(ramps,axis=1)
        self.rampy_cube = np.squeeze(rampy_cube)

        # make reference cube for jumps detected with calwebb_detector1
        dq_ints = np.array_split(self.dq_3d_arr,len(self.dq_3d_arr),axis=0)
        dq_cube = np.concatenate(dq_ints,axis=1)
        self.dq_cube = np.squeeze(dq_cube) # bitwise cube with all the dq flags

        flag = 4 # jump detected flag
        jump_cube = np.full(self.dq_cube.shape, False, dtype=bool)

        for frame in range(len(self.dq_cube)):
            jump_arr = (self.dq_cube[frame] & flag) == flag
            jump_cube[frame] = jump_arr

        self.jump_cube = jump_cube # a boolean cube where True is for jumps detected
        

    def _circle_app(self,rad):
        """
        Makes a kinda circular aperture, probably not worth using. - from ryan
        """
        mask = np.zeros((int(rad*2+.5)+1, int(rad*2+.5)+1))
        c = rad
        x,y = np.where(mask==0)
        dist = np.sqrt((x-c)**2 + (y-c)**2)

        ind = (dist) < rad + .2
        mask[y[ind],x[ind]] = 1

        return mask
    

    def _mask_pixels(self,threshold = 47000): # could be udated w/ quality flags
        """
        returns a list of tuples that are pixel (row,col) coordinates
        that have masked out the non-science and saturated pixels
        """
        # load general miri mask
        mask = np.load('full_MIRI_mask.npy') # for full array

        # mask out pixels that get counts above threshold
        mask_sat = self.rampy_cube[-1] < threshold
        mask_sat = mask_sat.astype(int) # to convolve with aperture

        kernel = self._circle_app(5)

        mask_sat = convolve_fft(mask_sat, kernel)
        mask_sat = mask_sat >= 0.99 # boolean

        # creating a list of tuples which are the (row,column) coords of each science pixel
        rows = list(range(self.rampy_cube.shape[1]))
        cols = list(range(self.rampy_cube.shape[2]))

        pixels = []

        for i in rows:
                row_num = [i] * len(cols)
                pixel_row = list(zip(row_num,cols)) # tuples of a single row's (i's) pixel coordinates
                pixels.extend(pixel_row)

        self.mask_tot = mask_sat & mask
        nan_mask = self.mask_tot * 1.0 
        nan_mask[nan_mask < 1] = np.nan
        self.nan_mask = nan_mask

        pixel_mask = self.mask_tot.flatten(order='C').tolist() # flattening mask to make same size/dimensions as the list of pixel coords
        self.masked_pixels = [pixel for pixel, m in zip(pixels, pixel_mask) if m]


    def _pixel_integration(self,cube,int_num,row,col):
        """
        Gets the x and y data of a specified integration for a specific pixel in a specified cube
        """
        integration_length = list(range(0,self.n_group))
        x_dat = [i + (int_num)*self.n_group for i in integration_length]
    
        ramp = []
        for x in x_dat:
            ramp.append(cube[x][row][col])

        return x_dat, ramp   


    def parallel_fit_df(self,cube,save_df=False,num_cores=20):
        """
        fits all ramps of specified cube parallely
        """
        fitting = Parallel(n_jobs=num_cores, verbose=0)(
            delayed(linear_fitting)(pixel,cube,self.n_int,self.n_group) for pixel in self.masked_pixels)

        df = pd.DataFrame(fitting, columns=["row","col","gradients","intercepts","residuals"])
        df['max_residual'] = df['residuals'].apply(max)

        self.df = df
        if save_df:
            self.df.to_csv('ramp fittings.csv', index=False)


    def _line(self,m,c,x):
        """
        straight line eqn lol
        """
        return [i*m + c for i in x]


    def _check_jump(self,coords):
        """
        checking if jump was detected in the dq cube
        """
        row, col = coords
        # check through all frames for jumps  
        vals = self.jump_cube[:, row, col]
        if vals.any():
            return 1, int(np.argmax(vals))  # 1 and first z index
        else:
            return 0, None


    def worst_fits(self, cube, num=15, require_jump=False):
        """
        Get worst fits by max_residual. Option to filter on jump status: (require_jump=True/False)
        """
        # add coords, jump, and jump_frame columns to df
        self.df['coords'] = self.df.apply(lambda row: (row['row'], row['col']), axis=1)
        self.df[['jump', 'jump_frame']] = self.df['coords'].apply(lambda c: pd.Series(self._check_jump(c)))

        # sort by residual
        self.worst_fits_df = self.df.sort_values(by='max_residual',ascending=False)

        # filter by if jump or not
        if require_jump is True:
            subdf = self.worst_fits_df[self.worst_fits_df['jump'] == 1].head(num).copy()
        elif require_jump is False:
            subdf = self.worst_fits_df[self.worst_fits_df['jump'] == 0].head(num).copy()
        else:
            subdf = self.worst_fits_df.head(num).copy()

        self.jump_fit_df = subdf

        rows = subdf['row'].values 
        cols = subdf['col'].values
        grads = subdf['gradients'].values
        cepts = subdf['intercepts'].values
        resids = subdf['residuals'].values

        plt.figure(constrained_layout=True, figsize=(16, 12))

        for i in range(len(cols)):
            ax = plt.subplot(5, 3, i + 1)
            row, col = rows[i], cols[i]

            for int_num in range(self.n_int):
                x, y = self._pixel_integration(cube, int_num, row, col)
                x, y = np.asarray(x, float), np.asarray(y, float)

                # remove first and last points
                x[[0, -1]] = np.nan
                y[[0, -1]] = np.nan

                mask = np.isfinite(y)
                x, y = x[mask], y[mask]

                # plot the data
                ax.scatter(x, y, c='tab:blue', s=5)

                # line fit
                m, c, r = grads[i][int_num], cepts[i][int_num], resids[i][int_num]
                y_line = self._line(m, c, x)
                ax.plot(x, y_line, label=f'r={r:.2f}')

            ax.set_title(f'Pixel ({row},{col}) | jump={subdf.iloc[i]['jump']}, frame={subdf.iloc[i]['jump_frame']}')
            ax.set_xlabel('Frame')
            ax.set_ylabel('Counts')
            ax.legend()

        plt.suptitle('Bad pixel ramps')
        plt.savefig('worst_fits.png', dpi=300)
        plt.show()


# --------------------- Image Search -----------------------


    def mega_inator(self,cube):
        """
        makes a mega cube out of a rampy one
        """
        frames = list(range(0,self.n_frame))
        integrations = list(range(0,self.n_int))
        int_frames = [i*self.n_group  for i in integrations]

        int_frames.pop(0) # in order to have list of indices of first frames in ramps except for the very first one
        end_frames = [((i+1)*self.n_group)-1 for i in integrations] # want to remove the frames with these indices

        mega_cube = np.zeros((self.n_frame,cube.shape[1],cube.shape[2]))
        difference = 0 # difference between end of one ramp and start of next
        zero_ff = 0 # fudge factor to zero the ramps

        for frame in frames:
            if frame == 0:
                zero_ff = cube[frame]
                mega_cube[frame] = cube[frame] - zero_ff
            elif frame in int_frames:
                zero_ff = cube[frame] 
                difference = mega_cube[frame-2] + 2*(mega_cube[frame-2] - mega_cube[frame-3])
                mega_cube[frame] = cube[frame] + difference - zero_ff
            else:
                mega_cube[frame] = cube[frame] + difference - zero_ff

        # recalac int_frames
        int_frames = [i * self.n_group for i in integrations]
        end_frames = [(i+1) * self.n_group -1 for i in integrations]
        int_frames.extend(end_frames)

        mega_cube_masked = mega_cube.copy()
        nans_frame = np.full_like(mega_cube_masked[0], np.nan)

        for frame in int_frames:
            mega_cube_masked[frame] = nans_frame

        self.mega_cube = mega_cube
        self.mega_cube_masked = mega_cube_masked


    def _cube_gradient(self,cube):
        """
        make a gradient cube with the fakey fake frames - to be done
        """
        fakeified_cube = cube.copy()
        for int_num in range(self.n_int-1): # for all integrations but the last one
            val = (int_num+1)*self.n_group
            fakeified_cube[val-1] = 2*fakeified_cube[val-2] - fakeified_cube[val-3] # last frame of the integration
            fakeified_cube[val] = 3*fakeified_cube[val-2] - 2*fakeified_cube[val-3] # first frame of the next integration

        self.fakey_cube = fakeified_cube

        self.grad_cube = np.gradient(fakeified_cube,axis=0)


    def _reference_frame(self):
        """
        makes a reference frame from median of useful images
        """
        fake_frames = []
        for int_num in range(self.n_int-1):
            fake_frames.append((int_num+1)*self.n_group-1)

        no_nans = np.nansum(self.grad_cube,axis=(1,2)) > 0

        for frame in fake_frames:
            no_nans[frame] = False

        # masking out the all nan frames and the fake frames
        good_slices = self.grad_cube.copy()[no_nans]

        self.med_frame = np.nanmedian(good_slices,axis=0)


    def _cube_differenced(self): # split up to deal with nan frames
        """
        make a differenced cube from gradient cube using a median frame as reference- now for fakey fake frames - to be done
        """
        diff_cube = np.zeros_like(self.grad_cube)

        for frame in range(2,self.n_frame-2):
            diff_cube[frame] = self.grad_cube[frame] - self.med_frame

        for int_num in range(self.n_int):
            diff_cube[(int_num+1)*self.n_group-1] = np.zeros_like(self.grad_cube[0])

        self.diff_cube = diff_cube
        self.diff_cube_masked = self.diff_cube.copy() * self.mask_tot


    def _psf_kernel(self,size=10):
        """
        creates kernel based on filter
        """
        miri = stpsf.MIRI()
        miri.filter = self.filter
        psf = miri.calc_psf(fov_pixels=size)
        self.kernel = psf[3].data


    def source_finding(self,source_sigma: float = 12,plot=True,cut=True):
        """
        detecting transients using *StarFinder* on differenced cube
        skips all-NaN frames
        """
        frame_num = list(range(0, self.n_frame))
        self.source_df = pd.DataFrame()

        for frame in frame_num:
            data = self.diff_cube[frame].copy()
            neg_data = (-1)*data
            combined_df = pd.DataFrame()

            if np.isnan(data).all():
                print(f"Skipping frame {frame}: (oops) all NaNs.")

            _, med, std = sigma_clipped_stats(neg_data.copy())
            Finder = StarFinder(med + source_sigma * std, self.kernel)
            stars_neg = Finder(neg_data.copy())

            _, med, std = sigma_clipped_stats(data.copy())
            Finder = StarFinder(med + source_sigma * std, self.kernel)
            stars_pos = Finder(data.copy())

            if plot:
                plt.figure()
                plt.title(f'Frame {frame}')
                plt.imshow(data, vmin=-100, vmax=100)
                plt.colorbar()

            masked_set = set(self.masked_pixels)

            if stars_pos is not None and len(stars_pos) > 0:
                stars_pos_df = stars_pos.to_pandas()
                stars_pos_df['frame'] = frame
                stars_pos_df['sign'] = 1
                stars_pos_df['coords'] = stars_pos_df.apply(lambda row: (int(round(row['ycentroid'])), int(round(row['xcentroid']))), axis=1)
                stars_pos_df['not_masked'] = ~stars_pos_df['coords'].isin(masked_set)
                stars_pos_df['buffer'] = ((stars_pos_df['xcentroid'] > 1016) | (stars_pos_df['xcentroid'] < 16) |
                                            (stars_pos_df['ycentroid'] > 1012) | (stars_pos_df['ycentroid'] < 12))
                stars_pos_df['good_fwhm'] = (stars_pos_df['fwhm'] > 3) & (stars_pos_df['fwhm'] < 6) # FHWM cut
                combined_df = pd.concat([combined_df, stars_pos_df])

                if plot: # plot only sources not in mask region
                    if cut:
                        masked_pos = stars_pos_df[(~stars_pos_df['not_masked'])&(~stars_pos_df['buffer'])&(stars_pos_df['good_fwhm'])]
                        plt.scatter(masked_pos['xcentroid'], masked_pos['ycentroid'],
                                   marker='*', edgecolor='r', facecolor='none', s=60, label='+')
                    else:
                        plt.scatter(stars_pos_df['xcentroid'], stars_pos_df['ycentroid'],
                                   marker='*', edgecolor='r', facecolor='none', s=60, label='+')
            else:
                print(f'No positive sources detected in frame {frame}')

            if stars_neg is not None and len(stars_neg) > 0:
                stars_neg_df = stars_neg.to_pandas()
                stars_neg_df['frame'] = frame
                stars_neg_df['sign'] = -1
                stars_neg_df['coords'] = stars_neg_df.apply(lambda row: (int(round(row['ycentroid'])), int(round(row['xcentroid']))), axis=1)
                stars_neg_df['not_masked'] = ~stars_neg_df['coords'].isin(masked_set)
                stars_neg_df['buffer'] = ((stars_neg_df['xcentroid'] > 1016) | (stars_neg_df['xcentroid'] < 16) |
                                            (stars_neg_df['ycentroid'] > 1012) | (stars_neg_df['ycentroid'] < 12))
                stars_neg_df['good_fwhm'] = (stars_neg_df['fwhm'] > 3) & (stars_neg_df['fwhm'] < 6)
                combined_df = pd.concat([combined_df, stars_neg_df])

                if plot: # plot only sources not in mask region
                    if cut:
                        masked_neg = stars_neg_df[(~stars_neg_df['not_masked'])&(~stars_neg_df['buffer'])&(stars_neg_df['good_fwhm'])]
                        plt.scatter(masked_neg['xcentroid'], masked_neg['ycentroid'],
                                    marker='*', edgecolor='white', facecolor='none', s=60, label='-')
                    else:
                        plt.scatter(stars_neg_df['xcentroid'], stars_neg_df['ycentroid'],
                                    marker='*', edgecolor='white', facecolor='none', s=60, label='-')
            else:
                print(f'No negative sources detected in frame {frame}')

            self.source_df = pd.concat([self.source_df,combined_df])

            if plot:
                plt.gca().invert_yaxis()
                plt.legend()

        # filtering the detections so have df of sources in science region, not near edge of detector,
        # and have a 'good' fhwm, that is greater than 3 and less than 6 currently
        filtered_source_df = self.source_df[(~self.source_df['not_masked'])&(~self.source_df['buffer'])&(self.source_df['good_fwhm'])]
        self.filtered_source_df = filtered_source_df


    def _ap_photometry(self,row):
        """
        aperture photometry on the filtered dataframe of detected sources
        """
        image = self.diff_cube_masked[row['frame']]
        aperture_radius = row['fwhm'] * 1.5 # 1.5x fwhm for radius of aperture
        annulus_r_in = 4 * aperture_radius 
        annulus_r_out = 6 * aperture_radius 
        aperture = CircularAperture( (row['xcentroid'], row['ycentroid']), aperture_radius )
        phot_table = aperture_photometry(image, aperture)
        phot_table['aperture_sum'][0]


        # identify background sample
        ys, xs = np.indices(image.shape)
        dist = np.hypot(xs - row['xcentroid'], ys - row['ycentroid'])
        isannulus = (dist > annulus_r_in) & (dist < annulus_r_out)
        bgimage = np.empty(image.shape) + np.nan
        bgimage[isannulus] = image[isannulus]
        medbg = np.median(image[isannulus])


        counts = phot_table['aperture_sum'][0] - medbg * aperture.area
        return counts

    
    def _counts_to_df(self):
        """
        add the counts from aperture photometry to the filtered dataframe
        """
        self.filtered_source_df['counts'] = self.filtered_source_df.apply(self._ap_photometry,axis=1)


# --------------------- StarFinder Avoidance Functions -----------------------


    def _cube_significance(self,magic_number=3):
        """
        making a significance cube - dividing the differenced cube by the 
        standard deviation of the background of each frame
        """
        frame_num = list(range(0, self.n_frame))
        sig_cube = np.zeros_like(self.diff_cube)

        for frame in frame_num:
            data = self.diff_cube_masked[frame].copy()
            _, med, std = sigma_clipped_stats(data.copy())

            sig_cube[frame] = (self.diff_cube_masked[frame]-med) / std
        
        self.sig_cube = sig_cube
        self.bool_sig_cube = sig_cube > magic_number


    def _cube_threshold(self,rad=2,threshold=9):
        """
        convolves the significance cube with a circle and identifies
        the bits above a threshold, above which should be psf-like sources
        and below are cosmic ray junk stuffs
        """
        frame_num = list(range(0, self.n_frame))
        conv_sig_cube = np.zeros_like(self.sig_cube)
        bool_threshold_cube = np.zeros_like(self.sig_cube)

        for frame in frame_num:
            conv_sig_cube[frame] = convolve_fft(self.bool_sig_cube[frame],self._circle_app(rad),normalize_kernel=False)

            bool_threshold_cube[frame] = conv_sig_cube[frame] > threshold

        self.conv_sig_cube = conv_sig_cube
        self.bool_threshold_cube = bool_threshold_cube


    def _cube_rolling_sum(self,num_frames=4,threshold=3):
        """
        rolling sum over (num_frames) frames of threshold cube, cut for >= threshold
        """
        rolling_sum_cube = np.zeros_like(self.bool_threshold_cube, dtype=int)

        for frame in range(self.n_frame):
            rolling_sum_cube[frame] = np.sum(self.bool_threshold_cube[frame:frame+num_frames], axis=0)

        # make cut for >= threshold
        bool_rolling_sum_cube = rolling_sum_cube >= threshold

        self.rolling_sum_cube = rolling_sum_cube
        self.bool_rolling_sum_cube = bool_rolling_sum_cube



# --------------------- Plotting Functions -----------------------
        

    def plot_ramp(self,cube,coords,scatter=None,plot=None,clean=True,save=False):
        """
        plots the ramp of specified pixel in specified cube
        """
        row = coords[0]
        col = coords[1]
        
        plt.figure()
        for int_num in range(self.n_int):
            x, y = self._pixel_integration(cube, int_num, row, col)
            y = np.asarray(y, dtype=float)
            x = np.asarray(x, dtype=float)

            if clean: # remove first and last points
                y[0] = np.nan
                y[-1] = np.nan
                x[0] = np.nan
                x[-1] = np.nan

                mask = np.isfinite(y)
                x = x[mask]
                y = y[mask]

            # plot the data
            if scatter:
                plt.scatter(x, y, c='tab:blue', s=5)
            if plot:
                plt.plot(x, y, c='tab:blue')
        plt.title(f'Ramps for pixel {coords}')
        plt.xlabel('Frame')

        if save:
            plt.savefig('temp_ramp_title.png',dpi=600,bbox_inches='tight')

        plt.show()
        

    def plot_image(self,cube,frame,scale=None,save=False):
        """
        just a quicker way to plot the images
        """
        plt.figure()
        plt.title(f'Frame {frame}')

        if scale:
            plt.imshow(cube[frame],vmin=scale[0],vmax=scale[1],origin='lower')
        else:
            plt.imshow(cube[frame],origin='lower')

        plt.colorbar()

        if save:
            plt.savefig('temp_img_title.png',dpi=600,bbox_inches='tight')

        plt.show()

In [ ]:
test = Jurassic('pipeline_data/Obs/stage1/jw02107011001_02107_00001_mirimage_ramp.fits')